Sequential Chats and Customer Onboarding

## Setup

In [1]:
import os
api_key = os.environ["GEMINI_API_KEY"]
llm_config = { "model": "gemini-1.5-pro-latest", "api_key": api_key, "api_type": "google"}

In [2]:
from autogen import ConversableAgent

## Creating the needed agents

In [3]:

from autogen import ConversableAgent

onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",  # Removed spaces
    system_message='''You are a helpful customer service agent.
    You are here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)


## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [4]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our product. Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'name': '', 'location': ''}",
        },
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me what topics you are interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [5]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)



********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product. Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\Users\Pandiyan\anaconda3\envs\agentai\Lib\site-packages\autogen\agentchat\chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Pandi From Sivakasi


customer_proxy_agent (to onboarding_personal_information_agent):

Pandi From Sivakasi

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (41e76400-3349-48df-a81c-cf85368263f0): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_topic_preference_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
```json
{"name": "Pandi", "location": "Sivakasi"}
```


--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to onboarding_topic_preference_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Gospels from Bible


customer_proxy_agent (to onboarding_topic_preference_agent):

Gospels from Bible

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (2819b9fa-2d92-46ae-9380-7d1693136796): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to customer_engagement_agent):

Let's find something fun to read.
Context: 
```json
{"name": "Pandi", "location": "Sivakasi"}
```

Pandi from Sivakasi is interested in reading the Gospels from the Bible.


--------------------------------------------------------------------------------
customer_engagement_agent (to customer_proxy_agent):

Hey there, Pandi from Sivakasi!  So you're interested in the Gospels, that's fantastic!  It's a journey into some powerful stories and teachings. Since you're from Sivakasi, the "Printing Capita

## Print out the summary

In [6]:
summary=[]
for chat_result in chat_results:
    print(chat_result.summary)
    summary.append(chat_result.summary)
    print("\n")

{'content': '```json\n{"name": "Pandi", "location": "Sivakasi"}\n```\n', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'Pandi from Sivakasi is interested in reading the Gospels from the Bible.\n', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': "Pandi from Sivakasi, known for its printing industry, is interested in reading the Gospels (Matthew, Mark, Luke, and John). Each Gospel offers a unique perspective on Jesus' life.  A lighthearted joke about bicycles is shared.  The Gospels are compared to fireworks, illuminating important truths. A thought-provoking question is posed: Which Gospel writer would you most like to have chai with?  The conversation ends with encouragement for Pandi to enjoy reading the Gospels.\n", 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_c

In [7]:
summary

[{'content': '```json\n{"name": "Pandi", "location": "Sivakasi"}\n```\n',
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': None},
 {'content': 'Pandi from Sivakasi is interested in reading the Gospels from the Bible.\n',
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': None},
 {'content': "Pandi from Sivakasi, known for its printing industry, is interested in reading the Gospels (Matthew, Mark, Luke, and John). Each Gospel offers a unique perspective on Jesus' life.  A lighthearted joke about bicycles is shared.  The Gospels are compared to fireworks, illuminating important truths. A thought-provoking question is posed: Which Gospel writer would you most like to have chai with?  The conversation ends with encouragement for Pandi to enjoy reading the Gospels.\n",
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': Non

In [ ]:
#!pip install pandas

In [8]:
import pandas as pd
import ast
import json
import re

# Input string
#data_str = "{'name': 'Ramisha', 'location': 'Coimbatore'}"
data_str=summary[0]
print(data_str)
# Convert string to dictionary
# Extract JSON string using regex
match = re.search(r'```json\n(.*?)\n```', data_str['content'], re.DOTALL)
if match:
    json_str = match.group(1)  # Extract JSON part
    parsed_data = json.loads(json_str)  # Convert to dictionary
else:
    print("No valid JSON found!")
#data_dict = ast.literal_eval(parsed_data)

# Convert dictionary to DataFrame
df = pd.DataFrame([parsed_data])

df


{'content': '```json\n{"name": "Pandi", "location": "Sivakasi"}\n```\n', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


,name,location
0,Pandi,Sivakasi


In [9]:
summary[1]

{'content': 'Pandi from Sivakasi is interested in reading the Gospels from the Bible.\n',
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [10]:
df["TopicPreference"]=summary[1].get("content")

In [11]:
df

,name,location,TopicPreference
0,Pandi,Sivakasi,Pandi from Sivakasi is interested in reading t...


In [13]:
chat_results[2].chat_history[1].get("content")

'Hey there, Pandi from Sivakasi!  So you\'re interested in the Gospels, that\'s fantastic!  It\'s a journey into some powerful stories and teachings. Since you\'re from Sivakasi, the "Printing Capital of India",  I bet you appreciate a good book!  Did you know that the Gutenberg Bible, one of the first major books printed using movable type, was a Latin version of the Bible, including the Gospels?  Talk about printing history meeting biblical history!\n\nNow, since we are focusing on the Gospels (Matthew, Mark, Luke, and John), how about a little fun fact to get us started?  Each Gospel writer offers a unique perspective on Jesus\' life.  Matthew portrays Jesus as the fulfilled Jewish Messiah, Mark presents a fast-paced, action-packed account, Luke emphasizes Jesus\' compassion for the marginalized, and John shares a more theological and reflective perspective.  Think of it like four different friends telling the same story – each adding their own flavor!\n\nReady for a little chuckle 

In [14]:
df["Enagement"]=chat_results[2].chat_history[1].get("content")

In [15]:
df

,name,location,TopicPreference,Enagement
0,Pandi,Sivakasi,Pandi from Sivakasi is interested in reading t...,"Hey there, Pandi from Sivakasi! So you're int..."


In [ ]:
#!pip install gspread
#!pip install oauth2client

In [16]:
def add_leads_to_google_sheet(df):
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials

    # Define the scope for Google Sheets API
    scope = ['https://www.googleapis.com/auth/spreadsheets']

    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        'fair-subset-455717-s6-84dee767824c.json', scope
    )

    # Authorize the client
    client = gspread.authorize(creds)

    # Specify the Google Sheet ID
    sheet_id = '1gLkaCmWgQp_fuEqUeOkf-SsjwYfh4PuFQ2tpEKoLTFI'

    # Open the spreadsheet using its ID
    spreadsheet = client.open_by_key(sheet_id)

    # Select the first worksheet (index 0)
    worksheet = spreadsheet.get_worksheet(0)

    # Append each row of the DataFrame to the worksheet
    for _, row in df.iterrows():
        # Convert row to a list and append to the sheet
        worksheet.append_row(row.tolist())

    print("Data added successfully!")
    return "added"


In [17]:
add_leads_to_google_sheet(df)

Data added successfully!


'added'

## Print out the cost

In [18]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.00016375, 'gemini-1.5-pro-latest': {'cost': 0.00016375, 'prompt_tokens': 47, 'completion_tokens': 21, 'total_tokens': 68}}, 'usage_excluding_cached_inference': {'total_cost': 0}}


{'usage_including_cached_inference': {'total_cost': 0.00015999999999999999, 'gemini-1.5-pro-latest': {'cost': 0.00015999999999999999, 'prompt_tokens': 60, 'completion_tokens': 17, 'total_tokens': 77}}, 'usage_excluding_cached_inference': {'total_cost': 0.00015999999999999999, 'gemini-1.5-pro-latest': {'cost': 0.00015999999999999999, 'prompt_tokens': 60, 'completion_tokens': 17, 'total_tokens': 77}}}


{'usage_including_cached_inference': {'total_cost': 0.0033637499999999996, 'gemini-1.5-pro-latest': {'cost': 0.0033637499999999996, 'prompt_tokens': 611, 'completion_tokens': 520, 'total_tokens': 1131}}, 'usage_excluding_cached_inference': {'total_cost': 0.0033637499999999996, 'gemini-1.5-pro-latest': {'cost': 0.0033637499999999996, 'prompt_tokens': 611, 'co

In [19]:
for chat_result in chat_results:
    print(chat_result)
    print("\n")

ChatResult(chat_id=None, chat_history=[{'content': "Hello, I'm here to help you get started with our product. Could you tell me your name and location?", 'role': 'assistant', 'name': 'onboarding_personal_information_agent'}, {'content': 'Pandi From Sivakasi', 'role': 'user', 'name': 'customer_proxy_agent'}], summary={'content': '```json\n{"name": "Pandi", "location": "Sivakasi"}\n```\n', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}, cost={'usage_including_cached_inference': {'total_cost': 0.00016375, 'gemini-1.5-pro-latest': {'cost': 0.00016375, 'prompt_tokens': 47, 'completion_tokens': 21, 'total_tokens': 68}}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])


ChatResult(chat_id=None, chat_history=[{'content': 'Great! Could you tell me what topics you are interested in reading about?\nContext: \n```json\n{"name": "Pandi", "location": "Sivakasi"}\n```\n', 'role': 'assistant', 'name': 'onboar